In [1]:
import matplotlib.pyplot as plt 

In [2]:
import torch
from torch import nn 
from torch.nn import functional as F
from torch import optim

In [3]:
from dlc_practical_prologue import generate_pair_sets

In [4]:
import utils

In [5]:
tr_inp, tr_target, tr_classes, te_inp, te_target, te_classes = generate_pair_sets(1000)

In [6]:
# normalise data 
tr_inp, te_inp = utils.normalize(tr_inp, te_inp)

In [7]:
torch.mean(tr_inp)

tensor(8.6025e-09)

In [8]:
torch.bincount(tr_target)

tensor([458, 542])

In [9]:
# maybe fix class imbalance 

## Set model

In [10]:
# can try diff activation functions but for now using relu 

In [11]:
# also try dropout layers

In [12]:
from models import BaseNet
from utils import compute_class_accuracy, train_model

## Set params

In [13]:
criterion = nn.CrossEntropyLoss()

In [14]:
mini_batch_size = 100

In [15]:
nb_epochs = 10

In [16]:
eta = 1e-2

In [17]:
inp_channels = 2
out_class_channels = 2 
out_digit_channels = 10 

# Initially predict 0/1

In [18]:
model = BaseNet(inp_channels=inp_channels, out_channels=out_class_channels)

In [19]:
optimizer = optim.Adam(model.parameters(), lr=eta)

In [20]:
# mess around with optimiser params
# also try decaying learning rate

In [21]:
# DEAD KERNEL WHY 

In [ ]:
model, losses, tr_acc = train_model(model, criterion, tr_inp, tr_target,
                    mini_batch_size, optimizer, nb_epochs)

In [1]:
fig, ax = plt.subplots(nrows=1, ncols=2)
ax[0].plot(list(range(len(losses))), losses)
ax[0].set_xlabel("epoch")
ax[0].set_ylabel("cross entropy loss")
ax[1].plot(list(range(len(losses))), tr_acc)
ax[1].set_xlabel('epoch')
ax[1].set_ylabel("train acc")
plt.tight_layout()
plt.show()

NameError: name 'plt' is not defined

In [ ]:
te_acc = compute_class_accuracy(model, te_inp, te_target, mini_batch_size)

In [1365]:
te_acc

tensor(0.8090)